In [ ]:
# ===========================================
# 0. Устанавливаем зависимости (если нужны)
# (в Colab TF и sklearn уже есть, но на всякий случай)
!pip install -q scikit-learn seaborn tensorflow


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers, models
tf.random.set_seed(42)        # для воспроизводимости


url = 'https://raw.githubusercontent.com/37Degrees/DataSets/master/laptops.csv'
data = pd.read_csv(url, sep=';')
print('Размер:', data.shape)
display(data.head())

# ===========================================
# 3. Разделяем признаки/цель
X = data.drop('quality', axis=1).values
y = data['quality'].values.astype(np.float32)

# 4. Трен/валидация/тест
X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test,  y_val, y_test  = train_test_split(X_tmp, y_tmp, test_size=0.5, random_state=42)

# 5. Масштабируем признаки (важно для нейронок!)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val   = scaler.transform(X_val)
X_test  = scaler.transform(X_test)

# ===========================================
# 6. Определяем модель – нейронная сеть прямого распространения
model = models.Sequential([
    layers.Input(shape=(X_train.shape[1],)),    # 11 признаков
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='linear')        # регрессия → линейный выход
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

# 7. Раннее остановка, чтобы не переобучиться
early = tf.keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)

# 8. Обучаем
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=500, batch_size=32,
    callbacks=[early],
    verbose=0   # поставьте 1, если хотите видеть прогресс
)

# ===========================================
# 9. График обучения
plt.figure(figsize=(6,4))
plt.plot(history.history['loss'], label='train MSE')
plt.plot(history.history['val_loss'], label='val MSE')
plt.xlabel('Эпоха'); plt.ylabel('MSE'); plt.legend(); plt.title('Кривая обучения')
plt.show()

# ===========================================
# 10. Оценка на тесте
pred = model.predict(X_test).flatten()

mse = mean_squared_error(y_test, pred)
mae = mean_absolute_error(y_test, pred)
r2  = r2_score(y_test, pred)
print(f"MSE: {mse:.3f}  MAE: {mae:.3f}  R^2: {r2:.3f}")

# ===========================================
# 11. Scatter: факт vs. прогноз
plt.figure(figsize=(5,5))
plt.scatter(y_test, pred, alpha=0.6)
plt.xlabel('Фактическое качество'); plt.ylabel('Прогноз'); plt.title('Факт vs Прогноз')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--')
plt.show()

# ===========================================
# 12. Confusion Matrix после округления
y_test_int = y_test.astype(int)
pred_int   = np.rint(pred).astype(int)
cm = confusion_matrix(y_test_int, pred_int, labels=range(3,10))

plt.figure(figsize=(7,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=range(3,10), yticklabels=range(3,10))
plt.xlabel('Предсказано'); plt.ylabel('Истинное'); plt.title('Confusion Matrix (округлённые прогнозы)')
plt.show()


print('Статистика по ценам (в условных единицах):')
print(laptops['Price'].describe())

# Визуализируем распределение цен
plt.figure(figsize=(6,4))
sns.histplot(laptops['Price'], bins=30, kde=True)
plt.xlabel('Цена ноутбука')
plt.ylabel('Кол-во моделей')
plt.title('Распределение цен на ноутбуки')
plt.show()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 132936: invalid continuation byte